<a href="https://colab.research.google.com/github/Keer-Luo/ISYS5002-Assignment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Algorithm

1. User input ticker symbol
2. Use ticker to get profile page
3. Copied the table
4. For each row of the table
 * Process a row to extract pay
 * Find the last character to determin multiplier
 * Calculate salary, pay * multiplier
 * Calculate the income tax


Note: 
1. Need to validate the ticker, but for now I will assume the user type in a correct ticker
2. Still need to work out where I save the income tax to


In [5]:

#install requirements
!pip install kora -q

#load package page
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


# Use the ticker to get the URL

1. User input the ticker symbol

  NOTE: use the input function
 
2. get the URL（profile page)
 
 * set the example url
 * Find the url based on the ticker symbol

In [6]:
def build_url():
 # User input ticker symbol
 ticker_symbol = input("Please input ticker symbol ")
  
 print(ticker_symbol)
 # Use ticker to get profile page
 example_url = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
 result = example_url.format(ticker_symbol, ticker_symbol)
 return result


# Copied the table

1. check the elements in the webpage

2. Find what we need in the table

3. Get the cells of each row

3. read the table

In [7]:
def get_first_3_cells(row):
  cells = row.find_all('td')
  name = cells[0].text
  title = cells[1].text
  pay = cells[2].text
  return {'name': name, 'title':title, 'pay': pay}

In [8]:
def get_exec_records(soup):
  table = soup.find('table')
  body = table.find('tbody')
  rows = body.find_all('tr')
  
  exec_records =[]
  for row in rows:
   result = get_first_3_cells(row)
   exec_records.append(result) 
  return exec_records

# Calculate the salary and tax

In [9]:
# Calculate the salary
def get_salary(rec):
  
  pay = rec['pay']
  last_pay_character = pay[-1]
  num_part = pay[0:-1]

  if last_pay_character == 'M':
    salary = float(num_part) * 1000000
    #print("Millions")
  elif last_pay_character == 'k':
    salary = float(num_part) * 1000
    #print("Thousand")
  else:
    salary = 0
    #print("Not sure")

  return salary

In [10]:
def calculate_tax(income):
  # This function is to calculate the tax
   if income <= 18200:
    tax_payable = 0
   elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
   elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
   elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
   elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)


   return tax_payable

In [25]:
# Get the URL
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source, "html.parser")
records = get_exec_records(soup)

for record in records:
  result = get_salary(record)
  income_tax = calculate_tax(result)
  record['tax'] = income_tax

# creat a pandas datafrme (because recall in a earlier notebok a data frame is easy to save as csv)

execs_df = pd.DataFrame.from_records(records)
execs_df

Please input ticker symbol nab.ax
nab.ax


,name,title,pay,tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,N/A,0.0
9,Ms. Sally Mihell,Head of Investor Relations,N/A,0.0


# Save the data
They can choose if the data will be saved as a CSV 
and/or stored in an SQLite database


1. Ask the user if they want to save the table as a CSV file
 
2. Ask the user if they want to save the table in an SQLite database:
  * connect to the database
  * define the table name and create it in the database
  * Insert the dataFrame into the database
  * close the database
  * check the data

Note: Need the user to input their choice

In [22]:
save1 = input("Do you wnat to save the table as a CSV?")
if save1 == 'yes':
  import csv
  df = pd.DataFrame.from_records(records)
  df.to_csv(r'C:\Users\Ron\Desktop\records.csv')
  print("Done!")
elif save1 == 'no':
  print("OK!") 
else:
  print("Please input yes or no!")

Do you wnat to save the table as a CSV?yes
Done!


In [20]:
# I try to insert data into database after creating 'records.db', but I failed.
# There is so many questions with inserting codes
save2 = input("Do you wnat to save the table in an SQLite database?")
if save2 == 'yes':
  sqliteConnection = sqlite3.connect('records.db')
  #from sqlalchemy import create_engine
  #df.to_sql("records", engine, schema="SchemaName", if_exists="append", index=False)
  #con.close()
elif save1 == 'no':
  print("OK!") 
else:
  print("Please input yes or no!")

Do you wnat to save the table in an SQLite database?yes
